<h1>Coding envoriments</h1>

You may use Google Colab to write and run Python code on free GPUs provided by Colab. To do so, you'd need to create a Google Colab account with the following URL:

https://colab.research.google.com/#create=true

Then open a new notebook and name it as, e.g., COMP4600/5500-module1, which will be stored under your google drive.

However, working on Colab may have the following issues:

It may take too long to run code and API calls may be prone to failure (with a server response code of 503). It is better to install the Blueprints environment on your local machine and run Jupyter Notebook on your machine. To do so, follow the instructions given at

https://github.com/blueprints-for-text-analytics-python/blueprints-text

After installing miniconda3, you'd need to add the path to the conda executable to your existing $PATH and run activate as root. Should you run into installation problems, you may check for fixes by entering appropriate keywords on Google search--I am pretty sure that fixes to your problems have been posted by someone already.

Nevertheness, in most cases, working on Colab is convenient and meets most of our needs.

<b>Feel free to use any snippet in this and future classes for your coding assignments and project(s)</b>.

# Setup

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch01/setup.py')

%run -i setup.py

You are working on Google Colab.
Files will be downloaded to "/content".
!wget -P /content https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/settings.py
!wget -P /content/data/un-general-debates https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/data/un-general-debates/un-general-debates-blueprint.csv.gz
!wget -P /content/ch01 https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/ch01/requirements.txt

Additional setup ...
!pip install -r ch01/requirements.txt


# Load Python settings
Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [ ]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

ValueError: ignored

# Explore data

Text corpus are often organized as a DataFrame. A DataFrame is a 2d table with column attributes and row records, where a column could be numerical or non-numerical (alphabetical text)

The first task is to obtain some insights of the text corpus:

1.   Metadata -- Category (e.g., birthday, gender, height, age, SSN ...); Author of the text; Date/time the text is written/published; ...

2.   Content -- Words; Phrases; Entities; ...

3.   Scores -- Popularity; Sentiment; Readability; ...

**Dataset: the UN General Debate dataset**

In [ ]:
file = "un-general-debates-blueprint.csv"
file = f"{BASE_DIR}/data/un-general-debates/un-general-debates-blueprint.csv.gz" ### real location
df = pd.read_csv(file) ### the dataset is in CSV
pd.options.display.max_colwidth = 100 ### set display width
#pd.set_option("display.max_colwidth", -1) ### display the full text without truncation

# Take a peak at the data

In [ ]:
df.sample(2) ### select at random two records from df; different run would produce different outputs

,session,year,country,country_name,speaker,position,text
3847,51,1996,MDV,Maldives,Fathulla Jameel,Minister for Foreign Affairs,"﻿At the outset, allow me, Sir,\nto say how gratified we are to see you presiding over this\nAsse..."
2800,45,1990,IRQ,Iraq,KADRAT,NaN,﻿\nThe present statement was scheduled to be made before the General Assembly by the Permanent R...


In [ ]:
print(df.sample(2, random_state=50)) ### use random_state for reproducibility with a seed value


      session  year country country_name       speaker           position  \
5932       62  2007     SWE       Sweden  Anders Lidén  UN Representative   
578        30  1975     BWA     Botswana     Mr. MOGWE                NaN   

                                                                                                     text  
5932  Let me take this \nopportunity to express our strong support for the \npeople of Burma in their ...  
578   \n103 My delegation takes its place on this rostrum rather late in the day, when the representat...  


In [ ]:
df.columns

Index(['session', 'year', 'country', 'country_name', 'speaker', 'position',
       'text'],
      dtype='object')

In [ ]:
df.dtypes

session          int64
year             int64
country         object
country_name    object
speaker         object
position        object
text            object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7507 entries, 0 to 7506
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   session       7507 non-null   int64 
 1   year          7507 non-null   int64 
 2   country       7507 non-null   object
 3   country_name  7507 non-null   object
 4   speaker       7480 non-null   object
 5   position      4502 non-null   object
 6   text          7507 non-null   object
dtypes: int64(2), object(5)
memory usage: 410.7+ KB


In [ ]:
df.describe().T ### statistics of numerical columns, where T means transpose

,count,mean,std,min,25%,50%,75%,max
session,7507.00,49.61,12.89,25.00,39.00,51.00,61.00,70.00
year,7507.00,1994.61,12.89,1970.00,1984.00,1996.00,2006.00,2015.00


In [ ]:
df["country"].describe() ### statistics of non-numerical columns

count     7507
unique     199
top        ALB
freq        46
Name: country, dtype: object

In [ ]:
df[["country"]].describe().T

,count,unique,top,freq
country,7507,199,ALB,46


In [ ]:
df[["country", "speaker"]].describe().T

,count,unique,top,freq
country,7507,199,ALB,46
speaker,7480,5428,Seyoum Mesfin,12


In [ ]:
df['length'] = df['text'].str.len() 
### add a numerical column to count the number of characters of the text in each record
print(df['length'])

0       51419
1       29286
2       31839
3       26616
4       25911
        ...  
7502    10568
7503    25430
7504    13662
7505    14247
7506    11013
Name: length, Length: 7507, dtype: int64


In [ ]:
df[['length']].describe().T

,count,mean,std,min,25%,50%,75%,max
length,7507.00,17967.28,7860.04,2362.00,12077.00,16424.00,22479.50,72041.00


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
session,7507.00,49.61,12.89,25.00,39.00,51.00,61.00,70.00
year,7507.00,1994.61,12.89,1970.00,1984.00,1996.00,2006.00,2015.00
length,7507.00,17967.28,7860.04,2362.00,12077.00,16424.00,22479.50,72041.00


In [ ]:
where = ['country', 'speaker', 'position', 'text']
df[where].describe(include=np.object).T ### np.object can also be replaced with 'O' (big O)
### df[where].describe().T has the same effect

,count,unique,top,freq
country,7507,199,ALB,46
speaker,7480,5428,Seyoum Mesfin,12
position,4502,113,Minister for Foreign Affairs,1813
text,7507,7507,33: May I first convey to our President the congratulations of the Albanian delegation on his el...,1


# Cleanup data

Fill in missing data and unify names

In [ ]:
df.isna().sum() ### isna is the same as isnull

session            0
year               0
country            0
country_name       0
speaker           27
position        3005
text               0
length             0
dtype: int64

In [ ]:
df['speaker'].fillna('unknown',inplace=True); df['position'].fillna('unknown',inplace=True) 
### fill in null entries with 'unknown'
df.isna().sum()

session         0
year            0
country         0
country_name    0
speaker         0
position        0
text            0
length          0
dtype: int64

In [ ]:
pd.set_option("display.max_colwidth", -1)
df[df['speaker'].str.contains('Bush')] ### print the full text for entries that contain 'Bush' as speaker

session  year country   country_name             speaker   position  \
2720  44       1989  USA     United States  Bush                President   
3038  46       1991  USA     United States  George Bush         President   
4814  56       2001  USA     United States  George W. Bush      President   
5002  57       2002  USA     United States  Mr. George W. Bush  President   
5191  58       2003  USA     United States  George W. Bush      President   
5382  59       2004  USA     United States  Mr. George W Bush   President   
5760  61       2006  USA     United States  Mr. George W. Bush  President   
5951  62       2007  USA     United States  George W. Bush      President   
6143  63       2008  USA     United States  George W. Bush      President   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
df[df['speaker'].str.contains('Bush')]['speaker']

2720    Bush              
3038    George Bush       
4814    George W. Bush    
5002    Mr. George W. Bush
5191    George W. Bush    
5382    Mr. George W Bush 
5760    Mr. George W. Bush
5951    George W. Bush    
6143    George W. Bush    
Name: speaker, dtype: object

In [ ]:
df[df['speaker'].str.contains('Bush')]['speaker'].value_counts() ### multiple forms of writing Bush

George W. Bush        4
Mr. George W. Bush    2
Bush                  1
George Bush           1
Mr. George W Bush     1
Name: speaker, dtype: int64

In [ ]:
number_of_rows = len(df.index)
for i in range(number_of_rows): ### Unify all appearances of Bush to George W. Bush
    if 'Bush' in df['speaker'][i]:
        df['speaker'][i] = 'George W. Bush'

In [ ]:
df[df['speaker'].str.contains('Bush')]['speaker'].value_counts()

George W. Bush    9
Name: speaker, dtype: int64

# Tokenization with Regular Expressions

Tokenization is the process of extracting words from a sequence of characters. Python's native method str.split() tokenizes words from a word string, separated by whitespace. Sometimes we may want to use regular expressions to obtain tokens that match more specific patterns, such as to obtain tokens that must contain at least one Unicode letter.

Python library regex provides many ways to represent string patterns. We use \p{L} to select Unicode letters, which cover accented letters. For example, the following regular expression matches tokens consisting of at least one Unicode letter, preceded and followed by an arbitary sequence of alphanumeric characters (digits, letters, underscore, and hyphens), where \w covers digits, letters, and underscore, and \\' covers a single quote.

More about Python's regular expressions will be covered in later modules.

In [ ]:
text = "Let's defeat COVID-19 together in 2022!"
text.split()

["Let's", 'defeat', 'COVID-19', 'together', 'in', '2022!']

In [ ]:
import regex as re
def tokenize(text): ### tokenization
    return re.findall(r'[\w-\']*\p{L}[\w-\']*', text)
tokens = tokenize(text)
print(tokens)
print("|".join(tokens))

["Let's", 'defeat', 'COVID-19', 'together', 'in']
Let's|defeat|COVID-19|together|in


# Stop words

Stop words are determiners, auxiliary verbs, adverbs, and so on. Stop words usually don't carry much information and so they are often removed for analyzing the underlying text. Stop word removal, however, is a coarse-grained rule-based method. Be careful with the stop word lists you're using. 


In [ ]:
import nltk
import pprint ### pretty print
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))
pp = pprint.PrettyPrinter(width=120, compact=True)
pp.pprint(stopwords) ### a set of stopwords from NLTK

{'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn',
 "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't",
 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn',
 "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn',
 "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or',
 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's",
 'should', "should've", 'shouldn', "shouldn't", 'so',

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jie_wang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Add new stop words to an existing list of stop words

In [ ]:
include_stopwords = {'dear', 'regards', 'must', 'would', 'also'}
exclude_stopwords = {'against'}

stopwords |= include_stopwords
stopwords -= exclude_stopwords

In [ ]:
def remove_stop(tokens): ### return words whose lower case is not in the set of stop words
    return [t for t in tokens if t.lower() not in stopwords]

pipeline = [str.lower, tokenize, remove_stop] ### Preprocessing pipeline of methods

def prepare(text, pipeline):
    tokens = text
    for method in pipeline:
        tokens = method(tokens)
    return tokens 

In [ ]:
df['tokens'] = df['text'].progress_apply(prepare, pipeline=pipeline) 
### insert a new column of tokens as a list for each speech in DataFrame df

  0%|          | 0/7507 [00:00<?, ?it/s]

In [ ]:
df.sample(4, random_state=2)

session  year country      country_name      speaker  \
2689  44       1989  PNG     Papua New Guinea   SOMARE       
3738  50       1995  ZAF     South Africa      Mr. Nzo       
2328  42       1987  GRC     Greece             PAPOULIAS    
4546  55       2000  KHM     Cambodia          Hor Namhong   

                          position  \
2689  unknown                        
3738  Minister for Foreign Affairs   
2328  unknown                        
4546  Minister for Foreign Affairs   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

# Term Frequency (TF)

Count the appearences of each token (word frequency) to form a bag of words.

In [ ]:
df['num_tokens'] = df['tokens'].progress_map(len) 
### count the total number of different tokens, not frequency
df['num_tokens']

  0%|          | 0/7507 [00:00<?, ?it/s]

0       4129
1       2330
2       2547
3       2137
4       2027
        ... 
7502    880 
7503    2089
7504    1145
7505    1184
7506    896 
Name: num_tokens, Length: 7507, dtype: int64

In [ ]:
from collections import Counter

tokens = tokenize("She likes my cats and my cats like my sofa.")

counter = Counter(tokens) ### count frequency
print(counter)

more_tokens = tokenize("She likes dogs and cats")
counter.update(more_tokens) ### update frequency
print(counter)

Counter({'my': 3, 'cats': 2, 'She': 1, 'likes': 1, 'and': 1, 'like': 1, 'sofa': 1})
Counter({'my': 3, 'cats': 3, 'She': 2, 'likes': 2, 'and': 2, 'like': 1, 'sofa': 1, 'dogs': 1})


In [ ]:
df['tokens'].progress_map(counter.update) ### count token frequency for each speech in df
pp.pprint(counter.most_common(10)) ### the top 10 most frequent words and their counts

  0%|          | 0/7507 [00:00<?, ?it/s]

[('nations', 124307), ('united', 120705), ('international', 117142), ('world', 87245), ('countries', 85342),
 ('peace', 72028), ('development', 62503), ('states', 59556), ('security', 58323), ('people', 54054)]


In [ ]:
def count_words(df, column='tokens', preprocess=None, min_freq=2): ### counter as a DataFrame
    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into a DataFrame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'

    return freq_df.sort_values('freq', ascending=False)

freq_df = count_words(df)
freq_df.head(10)

,freq
token,
nations,124307
united,120705
international,117142
world,87245
countries,85342
peace,72028
development,62503
states,59556
security,58323


In [ ]:
count_words(df, column='text', preprocess=lambda text: re.findall(r'\w{22,}', text)) 
### long words of length at least 22

,freq
token,
counterrevolutionaries,7


# Inverse Document Frequency (IDF)

IDF of a token $t$ with respect to a corpus $D$ of documents is defined by
$$IDF(t) = \log\frac{N+1}{|\{d \in D: t \in d\}|},$$
where $N$ is the total number of documents in the corpus and $d$ is a document in the corpus.

The TF-IDF of a token $t$ is defined by 
$TF(t) \times IDF(t)$, which is a better measure of the sailience of $t$ than just using term frequency $TF(t)$ alone. 

In [ ]:
def compute_idf(df, column='tokens', preprocess=None, min_df=2):

    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(set(tokens))

    # count tokens
    counter = Counter()
    df[column].progress_map(update)

    # create data frame and compute idf
    idf_df = pd.DataFrame.from_dict(counter, orient='index', columns=['df'])
    idf_df = idf_df.query('df >= @min_df')
    idf_df['idf'] = np.log(len(df)+1/idf_df['df'])
    ### +1 makes sure that tokens appearning in all documents isn't ignored
    idf_df.index.name = 'token'
    return idf_df

In [ ]:
idf_df = compute_idf(df)

  0%|          | 0/7507 [00:00<?, ?it/s]

In [ ]:
freq_df['tfidf'] = freq_df['freq'] * idf_df['idf']
freq_df.sample(2)

,freq,tfidf
token,,
ceres,2,17.85
indispensability,26,232.01


In [ ]:
# not in book: for more data: joining is faster
freq_df = freq_df.join(idf_df)
freq_df['tfidf'] = freq_df['freq'] * freq_df['idf']

# Coding Assignment 1

In the 2015 USA speech, count the number of appearances of "United Nations", replace it with "UN", and put the modifed text back to df. 

<i>Hint</i> 
<li> To see how to merge elements in a list, see https://www.geeksforgeeks.org/python-merge-list-elements/ 
    
<li>    The following snippet produces the desired text into a list of tokens
<code>
text = df.query("year==2015 and country=='USA'")['text'].values[0] 
text_list = list(text.split(" "))
</code>

In [ ]:
text = list(df.query("year==2015 and country=='USA'")['text'].values[0].split(" "))
new_text = [] # Creates the new list for word replacements.

i = 0
while(i < len(text)):

  if(text[i] == "United" and text[i + 1] == "Nations"): #Checks to see if the two words next to each other are: "United" and "Nations"
    new_text.append("UN") # Replaces said word with "UN"
    i += 1
  else:
    new_text.append(text[i]) #Inserts original word
  i += 1

counts = {word: 0 for word in new_text} #Initializes a map for counting of the corpus

# Counts the corpus
for word in new_text:
  counts[word] += 1

print(f"USA Speach ''United Nations'' words count == {counts['UN']},\nnew text:=\n'{' '.join(new_text)}'")
df.query("year==2015 and country=='USA'")['text'].values[0] = ' '.join(new_text)

USA Speach ''United Nations'' words count == 8,
new text:=
'Seventy years after the founding of the UN it is worth reflecting on what the members of this body, working together, have helped to achieve. Out of the ashes of the Second World War, having witnessed the unthinkable power of the atomic age, the United States has worked with many nations in the Assembly to prevent a third world war — by forging alliances with old adversaries; by supporting the steady emergence of strong democracies accountable to their people and not to any foreign Power; and by building an international system that imposes a cost on those who choose conflict over cooperation, an order that recognizes the dignity and equal worth of all people.
That has been the work of seven decades. That is the ideal that this body has, at its best, pursued. Of course, there have been too many times when, collectively, we have fallen short of those ideals. Over the seven decades, terrible conflicts have claimed untold victims

# Finding Keywords in Context (FWIC)

In [ ]:
import textacy

if textacy.__version__ < '0.11': # standard usage
    from textacy.text_utils import KWIC
    
else: # for textacy 0.11.x
    from textacy.extract.kwic import keyword_in_context

    def KWIC(*args, **kwargs):
        # call keyword_in_context with all params except 'print_only'
        return keyword_in_context(*args, 
                           **{kw: arg for kw, arg in kwargs.items() 
                            if kw != 'print_only'})

ValueError: ignored

# New Section

In [ ]:
def kwic(doc_series, keyword, window=40, print_samples=5):

    def add_kwic(text):
        kwic_list.extend(KWIC(text, keyword, ignore_case=True, 
                              window_width=window, print_only=False))

    kwic_list = []
    doc_series.progress_map(add_kwic)

    if print_samples is None or print_samples==0:
        return kwic_list
    else:
        k = min(print_samples, len(kwic_list))
        print(f"{k} random samples out of {len(kwic_list)} " + \
              f"contexts for '{keyword}':")
        for sample in random.sample(list(kwic_list), k):
            print(re.sub(r'[\n\t]', ' ', sample[0])+'  '+ \
                  sample[1]+'  '+\
                  re.sub(r'[\n\t]', ' ', sample[2]))

In [ ]:
random.seed(22) ###
kwic(df[df['year'] == 2015]['text'], 'sdgs', print_samples=5)

AttributeError: ignored

In [ ]:
random.seed(22) ###
kwic(df[df['year'] == 2015]['text'], 'sids', print_samples=5)

In [ ]:
random.seed(20) ###
kwic(df[df['year'] == 2015]['text'], 'sam', print_samples=10)

# N-Grams and Phrases

It is often more useful to work on phrases. An easy way to phraserize text is to first identify bigrams or trigrams (a phrase is often formed by two or three words), then remove stop words from them.

In [ ]:
def ngrams(tokens, n=2, space=' '):
    return [space.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]
    ### zip(*[tokens[i:] for i in range(n)]) means to zip tokens[0:], ... tokens[n-1:] together to 
    ### generate a list of n-grams. You may uncomment the following and run it to see what happens
#text = 'this is a test.'
#tokens = tokenize(text)
#bigrams = set(zip(*[tokens[i:] for i in range(2)]))
#print(bigrams)
#bigrams = [' '.join(x) for x in bigrams]
#print(bigrams)

text = "the visible manifestation of the global climate change"
tokens = tokenize(text)
print("|".join(ngrams(tokens)))
print("|".join(ngrams(tokens,3)))

the visible|visible manifestation|manifestation of|of the|the global|global climate|climate change
the visible manifestation|visible manifestation of|manifestation of the|of the global|the global climate|global climate change


In [ ]:
def ngrams(tokens, n=2, space=' ', stopwords=set()):
    return [space.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])
            if len([t for t in ngram if t in stopwords])==0] ### if ngram doesn't contain stop words

print("Bigrams:", "|".join(ngrams(tokens, 2, stopwords=stopwords)))
print("Trigrams:", "|".join(ngrams(tokens, 3, stopwords=stopwords)))

Bigrams: visible manifestation|global climate|climate change
Trigrams: global climate change


In [ ]:
df["bigrams"] = df['text'].progress_apply(prepare, pipeline=[str.lower, tokenize])\
                          .progress_apply(ngrams, n=2, stopwords=stopwords) ### Add a column for bigrams in df
count_words(df, "bigrams").head(5) ### the top 5 most frequent bigrams

  0%|          | 0/7507 [00:00<?, ?it/s]

  0%|          | 0/7507 [00:00<?, ?it/s]

,freq
token,
united nations,103076
international community,27465
general assembly,26908
security council,20765
human rights,19825


In [ ]:
idf_df = compute_idf(df) ### re-initialize to be safe
# concatenate existing IDF data frame with bigram IDFs
idf_df = pd.concat([idf_df, compute_idf(df, 'bigrams', min_df=10)])

freq_df = count_words(df[df['year'] == 2015], 'bigrams')
freq_df['tfidf'] = freq_df['freq'] * idf_df['idf']

  0%|          | 0/7507 [00:00<?, ?it/s]

  0%|          | 0/7507 [00:00<?, ?it/s]

# Explanation on the zip function for n-grams

This explanation is based on
https://stackoverflow.com/questions/66203861/how-is-does-zip-generate-n-grams

The core idea is to zip together multiple versions of the same list where each of them starts from the next subsequent element to form n-grams.

For example, let L = ['A', 'B', 'C', 'D', 'E']. Then <code>zip(*[L[i:] for i in range(3)])</code> means that
L[0:] = L, L[1:] = ['B', 'C', ’D', 'E'], L[2:] = ['C', 'D', 'E'] get zipped as follows:

![zip.png](figures/zip.png)

In [ ]:
L = ['A','B','C','D', 'E']

print('original list: '.ljust(27),L[0:])
print('list skipping 1st element: ',L[1:])
print('list skipping 2 elements: '.ljust(27),L[2:])
print('bigram: '.ljust(27), list(zip(L,L[1:])))
print('trigram: '.ljust(27), list(zip(L,L[1:],L[2:])))

original list:              ['A', 'B', 'C', 'D', 'E']
list skipping 1st element:  ['B', 'C', 'D', 'E']
list skipping 2 elements:   ['C', 'D', 'E']
bigram:                     [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E')]
trigram:                    [('A', 'B', 'C'), ('B', 'C', 'D'), ('C', 'D', 'E')]


In [ ]:
[''.join(ngram) for ngram in zip(*[L[i:] for i in range(3)])]

['ABC', 'BCD', 'CDE']

# Compare Frequencies Across Time Intervals and Categories

In [ ]:
def count_keywords(tokens, keywords): 
    ### count the number of each keyword in the list of keywords appearing in the list of tokens
    tokens = [t for t in tokens if t in keywords]
    counter = Counter(tokens)
    return [counter.get(k, 0) for k in keywords]

In [ ]:
keywords = ['nuclear', 'terrorism', 'climate', 'freedom']
tokens = ['nuclear', 'climate', 'climate', 'freedom', 'climate', 'freedom']

print(count_keywords(tokens, keywords))

[1, 0, 3, 2]


In [ ]:
def count_keywords_by(df, by, keywords, column='tokens'):
    
    df = df.reset_index(drop=True) # if the supplied dataframe has gaps in the index
    freq_matrix = df[column].progress_apply(count_keywords, keywords=keywords)
    freq_df = pd.DataFrame.from_records(freq_matrix, columns=keywords)
    freq_df[by] = df[by] # copy the grouping column(s)
    
    return freq_df.groupby(by=by).sum().sort_values(by)

In [ ]:
freq_df = count_keywords_by(df, by='year', keywords=keywords)

  0%|          | 0/7507 [00:00<?, ?it/s]

In [ ]:
#pd.options.display.max_rows = 4

In [ ]:
#pd.options.display.max_rows = 60

In [ ]:
random.seed(23) ###
# analyzing mentions of 'climate' before 1980
kwic(df.query('year < 1980')['text'], 'climate', window=35, print_samples=5)

  0%|          | 0/1245 [00:00<?, ?it/s]

5 random samples out of 581 contexts for 'climate':
ation with a view to replacing the   climate   of fear, mistrust and hatred by to
ntributing to the maintenance of a   climate   of peace, harmony and understandin
ges in the international political   climate  . There is increasing awareness of 
rian-Israeli front, have created a   climate   which favors the continuation of n
 that, far from noting a promising   climate   in the most critical areas of inte
